In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pymysql
import emoji

In [19]:
# MySQL 데이터베이스에 연결하는 함수 정의
def conn(d_name):
    # MySQL과 파이썬을 연결해주는 라이브러리 불러오기
    import pymysql

    # MySQL 서버 주소 (보통 내 컴퓨터면 localhost)
    host_name = 'localhost'

    # MySQL 기본 포트 번호
    host_port = 3306

    # MySQL 사용자 이름
    username = 'root'

    # MySQL 비밀번호
    password = 'sqlsql123123'

    # 사용할 데이터베이스 이름 (함수 호출 시 전달받음)
    database_name = d_name

    # MySQL 서버에 실제로 연결하는 부분
    db = pymysql.connect(
        host=host_name,      # MySQL 서버 주소
        port=host_port,      # MySQL 서버 포트
        user=username,       # MySQL 사용자 이름
        password=password,  # MySQL 비밀번호
        database=database_name,  # 사용할 데이터베이스
        charset='utf8'       # 한글 깨짐 방지를 위한 설정
    )

    # 연결된 데이터베이스 객체를 반환
    return db

In [21]:
# beauty_shop 데이터베이스에 연결
db = conn('beauty_shop')

# SQL 문을 실행하기 위한 커서(cursor) 생성
cursor = db.cursor()

In [22]:
# 현재 MySQL 서버에 존재하는 모든 데이터베이스를 보여주는 SQL문
sql = "SHOW DATABASES"

# SQL 실행
cursor.execute(sql)

# 실행 결과를 모두 가져오기
result = cursor.fetchall()

# 결과 출력
result

(('beauty_shop',),
 ('ecommerce',),
 ('employees',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('sqldb',),
 ('student_mgmt',),
 ('sys',),
 ('tabledb',),
 ('world',))

In [23]:
# 사용할 데이터베이스를 beauty_shop으로 설정
sql = "USE beauty_shop"
cursor.execute(sql)

0

In [24]:
# 현재 세션에서 사용 중인 데이터베이스 이름을 확인하는 SQL
sql = "SELECT DATABASE()"

# SQL 실행
cursor.execute(sql)

# 결과 하나만 가져오기
result = cursor.fetchone()

# 결과 출력
result

('beauty_shop',)

In [25]:
# product 테이블 생성
sql = '''
CREATE TABLE IF NOT EXISTS product (
    PRODUCT_CODE int AUTO_INCREMENT NOT NULL,  -- 상품 코드 (자동 증가)
    TITLE VARCHAR(200) NOT NULL,                -- 상품명
    ORI_PRICE VARCHAR(100),                     -- 원래 가격
    DISCOUNT_PRICE VARCHAR(100),                -- 할인 가격
    link VARCHAR(200),                          -- 상품 링크
    PRIMARY KEY(PRODUCT_CODE)                   -- 기본 키 설정
);
'''

# 테이블 생성 SQL 실행
cursor.execute(sql)

# 변경 사항을 데이터베이스에 저장 (commit 안 하면 반영 안 됨)
db.commit()

In [26]:
# 현재 데이터베이스에 있는 테이블 목록 조회
sql = "SHOW TABLES"
cursor.execute(sql)
result = cursor.fetchall()
result

(('product',),)

In [27]:
# product 테이블 구조 확인
sql = "DESC product"
cursor.execute(sql)
result = cursor.fetchall()
result

(('PRODUCT_CODE', 'int', 'NO', 'PRI', None, 'auto_increment'),
 ('TITLE', 'varchar(200)', 'NO', '', None, ''),
 ('ORI_PRICE', 'varchar(100)', 'YES', '', None, ''),
 ('DISCOUNT_PRICE', 'varchar(100)', 'YES', '', None, ''),
 ('link', 'varchar(200)', 'YES', '', None, ''))

# 크롤링 코드 - insert 구문을 추가해서 변경

In [28]:
# =========================
# 1. 크롤링할 웹 페이지 요청
# =========================

# 크롤링할 대상 URL (토너/미스트 카테고리 페이지)
url = "http://jolse.com/category/toners-mists/1019/"

# 해당 URL에 요청을 보내고 응답 객체를 받음
html = urlopen(url)

# 응답 받은 HTML 문서를 바이트 형태로 읽기
htmls = html.read()

# BeautifulSoup 객체 생성 (HTML 파싱용)
bs_obj = BeautifulSoup(htmls, "html.parser")

In [29]:
# ==========================================================
# 2. 상품 1개(box)에서 정보 추출해서 dict로 반환하는 함수
#    - 제품명, 원가, 할인가, 상세페이지 링크 추출
#    - 데이터 전처리 포함
# ==========================================================
def get_product_info(box):

    # 상품 이름이 들어있는 strong 태그 찾기
    strong_tag = box.find("strong", {"class": "name"})

    # strong 태그의 텍스트에서 ':' 기준으로 나누어 제품명만 추출
    span = strong_tag.text.split(':', 1)[1]

    # 상품 상세 페이지로 이동하는 a 태그 찾기
    a = strong_tag.find("a")

    # 상세 페이지 링크 완성
    sub_link = 'https://jolse.com' + a["href"]

    # 링크에 포함된 이모지 제거
    sub_link = emoji.replace_emoji(sub_link, '')
    
    
    # =================
    # 가격 정보 추출
    # =================

    # 가격 정보가 들어있는 ul 태그 찾기
    price_ul = box.find("ul")

    # 원래 가격 (첫 번째 li)
    old_price = price_ul.select("li")[0]
    old_price = old_price.select("span")[1].text
    old_price = old_price.split(" ")[1]  # USD 제거

    # 할인 가격 (두 번째 li)
    dis_price = price_ul.select("li")[1]
    dis_price = dis_price.select("span")[1].text
    dis_price = dis_price.split(" ")[1]  # USD 제거
    
    
    
    # =================
    # 데이터 전처리
    # =================

    # SQL 에러 방지를 위해 작은따옴표(') 처리
    title = span.replace("'", "''")

    # 제품명에서 이모지 제거
    title = emoji.replace_emoji(title, '')

    # 할인 가격이 없는 경우 0.0으로 처리
    if dis_price == '':
        dis_price = '0.0'

    # 추출한 데이터를 딕셔너리 형태로 반환
    return {
        "prd_name": title,
        "price": old_price,
        "sale_price": dis_price,
        "sub_link": sub_link
    }

In [30]:
# ==================================================
# 3. 상품 정보를 DB에 저장하는 함수
# ==================================================
def save_data(prd_info):

    # INSERT SQL 문 작성
    sql = "INSERT INTO product (title, ori_price, discount_price, link) values('" \
        + prd_info["prd_name"] \
        + "'," \
        + prd_info['price'] \
        + "," \
        + prd_info['sale_price'] \
        + ",'" \
        + prd_info['sub_link'] \
        + "');"

    # SQL 문이 제대로 만들어졌는지 확인하고 싶으면 주석 해제
    # print(sql)

    # SQL 실행
    cursor.execute(sql)

In [31]:
# ==========================================================
# 4. 페이지 하나에서 모든 상품 정보를 가져오는 함수
# ==========================================================
def get_page_products(url):

    # 전달받은 URL 요청
    html = urlopen(url)

    # HTML 문서 읽기
    htmls = html.read()

    # BeautifulSoup 객체 생성
    bs_obj = BeautifulSoup(htmls, "html.parser")

    # ==========================
    # 상품 리스트 전체를 감싸는 ul 태그 찾기
    # ==========================
    ul = bs_obj.find("ul", {"class": "prdList grid5"})

    # ==========================
    # 상품 1개씩 들어있는 div 태그들 찾기
    # ==========================
    prd_boxes = ul.findAll("div", {"class": "description"})

    # ==========================
    # 각 상품 정보를 추출해서 DB에 저장
    # ==========================
    for box in prd_boxes:

        # 상품 1개 정보 추출
        prd = get_product_info(box)

        # 잘 추출되는지 확인하려면 주석 해제
        # print(prd)

        # DB에 상품 정보 저장
        save_data(prd)

# main 코드(프로그램 시작점)

In [32]:
# ==========================================
# 1. 상태바 표시를 위해 tqdm 라이브러리 불러오기
# ==========================================
from tqdm import tqdm  # 반복 진행 상황을 시각적으로 보여줌

# ==========================================
# 2. 크롤링할 URL 기본 주소
# ==========================================
url = "http://jolse.com/category/toners-mists/1019/?page="  # 페이지 번호를 뒤에 붙여서 사용

# ==========================================
# 3. 마지막 페이지 번호 확인 (실제 사이트에서는 아래 주석처럼 동적으로 확인 가능)
# ==========================================
# last = int(bs_obj.find("a",{"class":"last"})['href'].split("=")[1])
# 현재 사이트 상황에서는 13페이지가 마지막임
# 하지만 빠른 실습을 위해 2페이지까지로 가정
last = 2

# ==========================================
# 4. 1페이지부터 마지막 페이지까지 반복
# ==========================================
for i in tqdm(range(1, last+1)):  # tqdm으로 진행 상태 표시
    # 실제 요청할 URL 완성
    urlfin = url + str(i)

    # 해당 페이지의 모든 상품 정보를 가져와 DB에 저장
    get_page_products(urlfin)

100%|██████████| 2/2 [00:06<00:00,  3.05s/it]


In [33]:
# ==========================================
# 5. 지금까지 DB에 INSERT한 데이터 실제로 저장
# ==========================================
db.commit()  # commit 안 하면 저장 안 됨

In [34]:
# ==========================================
# 6. DB에 저장된 데이터 확인
# ==========================================
sql = "SELECT * FROM product"  # product 테이블 전체 조회
cursor.execute(sql)  # SQL 실행
result = cursor.fetchall()  # 모든 결과 가져오기
result[:5]  # 처음 5개만 확인

((1,
  ' B : Lab Essence Toner Pad 10ea/25g',
  '5.00',
  '1.99',
  'https://jolse.com/product/b-lab-essence-toner-pad-10ea25g/89277/category/1019/display/1/'),
 (2,
  ' EQQUALBERRY Swimming Pool Toner 300ml',
  '22.00',
  '15.40',
  'https://jolse.com/product/eqqualberry-swimming-pool-toner-300ml/87123/category/1019/display/1/'),
 (3,
  ' celimax Jiwoogae Heartleaf BHA Peeling Pad 60pcs (22AD)',
  '17.50',
  '13.12',
  'https://jolse.com/product/celimax-jiwoogae-heartleaf-bha-peeling-pad-60pcs-22ad/55940/category/1019/display/1/'),
 (4,
  ' SKIN1004 Madagascar Centella Probio-Cica Essense Toner 210ml',
  '24.00',
  '14.49',
  'https://jolse.com/product/skin1004-madagascar-centella-probio-cica-essense-toner-210ml/68504/category/1019/display/1/'),
 (5,
  ' Dr.Althea 345 Relief Cream Mist 100ml',
  '19.90',
  '13.93',
  'https://jolse.com/product/dralthea-345-relief-cream-mist-100ml/92356/category/1019/display/1/'))

In [35]:
# ==========================================
# 7. DB 연결 종료
# ==========================================
db.close()  # 연결을 종료해야 다른 프로그램에서 접근 가능

# db 테이블에 저장된 데이터 df로 가져오기

In [36]:
# ==========================================
# 1. 데이터베이스 연결
# ==========================================
db = conn("beauty_shop")  # beauty_shop DB에 연결

In [37]:
# ==========================================
# 2. product 테이블 전체 데이터를 DataFrame으로 가져오기
# ==========================================
sql = "SELECT * FROM product"  # SQL: product 테이블 전체 조회
df = pd.read_sql(sql, db)      # pandas로 읽어서 DataFrame으로 저장
df                              # DataFrame 확인 (전체 컬럼)

C:\Users\User\AppData\Local\Temp\ipykernel_22464\3828852949.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)      # pandas로 읽어서 DataFrame으로 저장


,PRODUCT_CODE,TITLE,ORI_PRICE,DISCOUNT_PRICE,link
0,1,B : Lab Essence Toner Pad 10ea/25g,5.00,1.99,https://jolse.com/product/b-lab-essence-toner-...
1,2,EQQUALBERRY Swimming Pool Toner 300ml,22.00,15.40,https://jolse.com/product/eqqualberry-swimming...
2,3,celimax Jiwoogae Heartleaf BHA Peeling Pad 60...,17.50,13.12,https://jolse.com/product/celimax-jiwoogae-hea...
3,4,SKIN1004 Madagascar Centella Probio-Cica Esse...,24.00,14.49,https://jolse.com/product/skin1004-madagascar-...
4,5,Dr.Althea 345 Relief Cream Mist 100ml,19.90,13.93,https://jolse.com/product/dralthea-345-relief-...
...,...,...,...,...,...
75,76,numbuzin No.1 Centella Re-Leaf Green Toner Pa...,24.00,16.80,https://jolse.com/product/numbuzin-no1-centell...
76,77,Isntree Hyaluronic Acid Toner Plus 200ml,22.20,17.76,https://jolse.com/product/isntree-hyaluronic-a...
77,78,APLB Glutathione Niacinamide Mist Essence 105ml,16.40,10.17,https://jolse.com/product/aplb-glutathione-nia...
78,79,Anua Azelaic 10 Hyaluron Redness Soothing Pad...,27.00,22.95,https://jolse.com/product/anua-azelaic-10-hyal...


In [38]:
# ==========================================
# 3. 특정 컬럼만 가져오기 (title, ori_price, discount_price)
# ==========================================
sql = "SELECT title, ori_price, discount_price FROM product"  # 원하는 컬럼만 조회
df = pd.read_sql(sql, db)  # pandas DataFrame으로 가져오기
df                           # DataFrame 확인 (선택한 컬럼만)

C:\Users\User\AppData\Local\Temp\ipykernel_22464\3871068375.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)  # pandas DataFrame으로 가져오기


,title,ori_price,discount_price
0,B : Lab Essence Toner Pad 10ea/25g,5.00,1.99
1,EQQUALBERRY Swimming Pool Toner 300ml,22.00,15.40
2,celimax Jiwoogae Heartleaf BHA Peeling Pad 60...,17.50,13.12
3,SKIN1004 Madagascar Centella Probio-Cica Esse...,24.00,14.49
4,Dr.Althea 345 Relief Cream Mist 100ml,19.90,13.93
...,...,...,...
75,numbuzin No.1 Centella Re-Leaf Green Toner Pa...,24.00,16.80
76,Isntree Hyaluronic Acid Toner Plus 200ml,22.20,17.76
77,APLB Glutathione Niacinamide Mist Essence 105ml,16.40,10.17
78,Anua Azelaic 10 Hyaluron Redness Soothing Pad...,27.00,22.95


In [39]:
# ==========================================
# 4. DB 연결 종료
# ==========================================
db.close()  # DB 연결 종료